In [28]:
from s3prl.downstream.mdd.dataset import L2ArcticDataset
from s3prl.downstream.augment_utils.add_noise import AddNoise
from IPython.display import Audio, display
import torch
import torchaudio
from blind_rt60 import BlindRT60
from matplotlib import pyplot as plt
import numpy as np
from tqdm import tqdm
import pandas as pd
from torch.utils.data import DataLoader

In [32]:
torch.manual_seed(1337)

In [46]:
dataset = L2ArcticDataset(
    "test",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/data/l2arctic_release_v5.0",
    "/home/xt0r3-user/cambridge/partii/dissertation/s3prl/s3prl/s3prl/downstream/mdd/data/",
)

loader = DataLoader(
           dataset,
            batch_size=1,
            shuffle=True,
        )

In [47]:
def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()
    display(Audio(waveform, rate=sample_rate))

In [48]:
# Create an instance of the BlindRT60 estimator
estimator = BlindRT60()
fs=16000
reverbs = []
for wav, *args in tqdm(loader):
    # Estimate the RT60
    rt60_estimate = estimator(wav[0], fs)
    reverbs.append(rt60_estimate)

    # # Visualize the results
    # fig = estimator.visualize(audio[1].numpy(), fs)
    # plt.show()

pd.Series(reverbs).describe()

 84%|████████▎ | 752/900 [1:46:10<20:53,  8.47s/it]  


KeyboardInterrupt: 

In [54]:
pd.read_json('test_reverbs.json', typ='series', orient='records').describe()

count    752.000000
mean       2.440593
std        0.698609
min        1.043862
25%        1.950407
50%        2.290579
75%        2.835344
max        6.221709
dtype: float64

In [ ]:
pd.read_json('train_reverbs.json', typ='series', orient='records').describe()

In [19]:
play_audio(audio[0], 16000)

In [ ]:
augmentation = AddNoise(
    prob=1.0,
    snr_low=0,
    snr_high=0.4,
    device="cpu",
)
audio = torch.nn.utils.rnn.pad_sequence(audio, batch_first=True)
lens = torch.tensor([len(audio[0]), len(audio[1])], dtype=torch.long)
aug_audio =     augmentation(
        audio,
        lens,
    )
play_audio(
    aug_audio[0],
    sample_rate=16000,
)

audio - aug_audio